In [1]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import pandas as pd
import numpy as np
import ast
import requests

In [2]:
df_clima = pd.read_csv("../Ficheros/ETL1.csv", index_col=0)
df_attacks = pd.read_csv("../Ficheros/attacks_limpieza_completa.csv", index_col = 0)

En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I.


In [3]:
df_attacks.head(1)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327


In [4]:
df_clima.head(1)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,paises
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 9}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 175, 'speed':...",26,2,6,1011,none,0,0,130,2,39.78373,-100.445882,USA


In [5]:
df_clima.paises.unique()

array(['USA', 'Australia', 'South Africa', 'New Zealand',
       'Papua New Guinea'], dtype=object)

In [6]:
df_clima.shape

(320, 19)

In [7]:
class Extraccion:
    
    # Constructor
    def __init__ (self, diccionario):
        
        self.diccionario = diccionario
    
    # Método para llamar a la API
    def llamar_API(self, producto, ruta):
        
        df_final= pd.DataFrame()

        for key, value in self.diccionario.items():
                
            url = f'http://www.7timer.info/bin/api.pl?lon=-{value[1]}&lat={value[0]}&product={producto}&output=json'
            response = requests.get(url)
            df= pd.DataFrame.from_dict(pd.json_normalize(response.json()["dataseries"]))
            df["latitud"] = value[0]
            df["longitud"] = value[1]
            df["paises"] = key
            df_final=pd.concat([df_final, df], axis= 0, ignore_index=True)
               
        df_final.to_csv(ruta)

In [8]:
class Limpieza:
    
    # Constructor
    
    def __init__(self, dataframe1, dataframe2): 
        
        self.dataframe1 = dataframe1
        self.dataframe2 = dataframe2
        
        
    # Método para poner en minúsculas los datos de las columnas categóricas 
       
    def minuscular(self, columna):
         
        self.dataframe2[columna]= self.dataframe2[columna].apply(lambda x:x.lower())
        
        return self.dataframe2
    
    
    # Método para filtrar por columnas según una lista de valores  
    def filtrar(self, columna, lista):
        
        self.dataframe1 = pd.DataFrame(self.dataframe1[self.dataframe1[columna].isin(lista)])  
       
        return self.dataframe1
    
    
    # Método para separar columnas con datos tipo json en distintas columnas  
    def separar(self, columna, valor1, valor2):   
        
        self.dataframe2[columna] = self.dataframe2[columna].apply(ast.literal_eval)

        x = self.dataframe2[columna].apply(pd.Series)      
        
        for values in range(len(x.columns)):
            nombre= valor1 + str(x[values].apply(pd.Series)[valor2][0])
            valores = list(x[values].apply(pd.Series)[valor1])
            self.dataframe2.insert(values, nombre, valores)
            
        return self.dataframe2
    
    # Método para group by    
    def agrupar_media(self, columna):
        
        self.dataframe2 = self.dataframe2.groupby(columna).mean().reset_index()
        
        return self.dataframe2
        
    
    # Método para unir dataframes          
    def unir(self, estrategia, columna1, columna2): 
        
        df = pd.merge(left= self.dataframe1, right = self.dataframe2, how= estrategia, left_on = columna1, right_on = columna2)
        
        return df
        
        
        

In [9]:
instancia = Limpieza(df_attacks, df_clima)


In [10]:
instancia.minuscular("paises")

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,paises
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 9}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 175, 'speed':...",26,2,6,1011,none,0,0,130,2,39.783730,-100.445882,usa
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 9}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 220, 'speed':...",25,2,6,1011,none,0,0,80,2,39.783730,-100.445882,usa
2,9,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 8}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 195, 'speed':...",25,2,5,1013,none,0,0,115,2,39.783730,-100.445882,usa
3,12,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 9}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 140, 'speed':...",25,2,5,1012,none,0,0,105,3,39.783730,-100.445882,usa
4,15,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 9}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 135, 'speed':...",24,2,5,1011,none,0,0,130,2,39.783730,-100.445882,usa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,180,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",26,2,11,1010,rain,3,0,100,3,-5.681607,144.248908,papua new guinea
316,183,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 9}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 100, 'speed':...",26,2,11,1010,rain,3,0,85,3,-5.681607,144.248908,papua new guinea
317,186,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 9}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 95, 'speed': ...",26,2,11,1012,rain,3,0,80,4,-5.681607,144.248908,papua new guinea
318,189,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 10}, {'layer': '900m...","[{'layer': '950mb', 'direction': 90, 'speed': ...",26,-1,12,1011,none,3,0,80,3,-5.681607,144.248908,papua new guinea


In [11]:
lista = ['usa', 'australia', 'south africa', 'new zealand','papua new guinea']

In [12]:
instancia.filtrar("country", lista)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.783730,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.783730,-100.445882,...,1,0,0,1,0,0,3,7,7,3.804860
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.783730,-100.445882,...,1,0,0,1,0,0,3,7,3,8.403629
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,...,1,0,0,1,0,0,3,7,7,6.313925
5,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,...,1,0,0,1,0,0,3,7,7,6.313925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6210,0,Unprovoked,south africa,27.0,Unespecific,Jul,Y,M,-28.816624,24.991639,...,0,0,1,0,0,1,3,6,7,6.313925
6212,0,Unprovoked,usa,27.0,Unespecific,Unknown,N,F,39.783730,-100.445882,...,1,0,0,1,0,0,3,14,7,6.313925
6213,0,Unprovoked,australia,27.0,Tiger shark,Unknown,Y,M,-24.776109,134.755000,...,0,0,1,0,0,1,2,14,7,6.313925
6214,0,Unprovoked,australia,27.0,Tiger shark,Unknown,Y,M,-24.776109,134.755000,...,0,0,1,0,0,1,2,14,7,6.313925


In [13]:
instancia.separar("rh_profile", "rh", "layer")

,rh950mb,rh900mb,rh850mb,rh800mb,rh750mb,rh700mb,rh650mb,rh600mb,rh550mb,rh500mb,...,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,paises
0,9,9,10,10,9,8,5,3,3,2,...,6,1011,none,0,0,130,2,39.783730,-100.445882,usa
1,9,10,9,8,7,4,3,4,4,4,...,6,1011,none,0,0,80,2,39.783730,-100.445882,usa
2,8,12,11,9,7,4,3,3,4,5,...,5,1013,none,0,0,115,2,39.783730,-100.445882,usa
3,9,13,13,15,6,3,3,2,3,4,...,5,1012,none,0,0,105,3,39.783730,-100.445882,usa
4,9,12,13,12,10,6,2,1,3,3,...,5,1011,none,0,0,130,2,39.783730,-100.445882,usa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,13,13,9,-2,0,2,-2,-4,-4,-4,...,11,1010,rain,3,0,100,3,-5.681607,144.248908,papua new guinea
316,9,13,15,-2,2,3,-3,-4,-4,-4,...,11,1010,rain,3,0,85,3,-5.681607,144.248908,papua new guinea
317,9,12,13,-1,3,0,-3,-4,-4,-4,...,11,1012,rain,3,0,80,4,-5.681607,144.248908,papua new guinea
318,10,11,7,-1,3,1,-3,-4,-4,-4,...,12,1011,none,3,0,80,3,-5.681607,144.248908,papua new guinea


In [14]:
instancia.separar("wind_profile", "speed", "layer")

,speed950mb,speed900mb,speed850mb,speed800mb,speed750mb,speed700mb,speed650mb,speed600mb,speed550mb,speed500mb,...,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,paises
0,2,2,3,3,3,3,3,2,3,3,...,6,1011,none,0,0,130,2,39.783730,-100.445882,usa
1,2,3,3,3,3,3,3,3,3,3,...,6,1011,none,0,0,80,2,39.783730,-100.445882,usa
2,2,3,3,3,3,2,2,3,3,3,...,5,1013,none,0,0,115,2,39.783730,-100.445882,usa
3,3,2,3,3,3,3,3,2,2,3,...,5,1012,none,0,0,105,3,39.783730,-100.445882,usa
4,2,2,2,3,3,3,3,2,3,3,...,5,1011,none,0,0,130,2,39.783730,-100.445882,usa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,4,4,4,3,3,3,3,3,3,4,...,11,1010,rain,3,0,100,3,-5.681607,144.248908,papua new guinea
316,4,5,4,3,3,3,3,3,3,3,...,11,1010,rain,3,0,85,3,-5.681607,144.248908,papua new guinea
317,5,5,4,3,3,3,2,3,3,3,...,11,1012,rain,3,0,80,4,-5.681607,144.248908,papua new guinea
318,5,4,4,3,3,3,2,3,3,3,...,12,1011,none,3,0,80,3,-5.681607,144.248908,papua new guinea


In [15]:
instancia.agrupar_media("paises")

,paises,speed950mb,speed900mb,speed850mb,speed800mb,speed750mb,speed700mb,speed650mb,speed600mb,speed550mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud
0,australia,3.062500,2.875000,2.781250,2.750000,2.859375,2.828125,2.812500,2.906250,3.109375,...,21.906250,2.484375,10.281250,1022.921875,2.125000,0.0,100.781250,2.937500,-24.776109,134.755000
1,new zealand,3.937500,3.828125,3.718750,3.718750,3.671875,3.812500,3.812500,3.921875,4.125000,...,11.062500,8.125000,10.687500,1024.390625,2.046875,0.0,199.140625,3.406250,-41.500083,172.834408
2,papua new guinea,4.093750,4.125000,3.765625,3.125000,2.625000,2.765625,2.734375,2.796875,3.187500,...,25.781250,2.218750,10.906250,1011.312500,1.937500,0.0,101.796875,3.515625,-5.681607,144.248908
3,south africa,3.546875,3.609375,3.562500,3.703125,3.984375,4.109375,4.515625,5.000000,5.625000,...,17.921875,1.703125,12.296875,1014.781250,4.437500,0.0,245.625000,3.187500,-28.816624,24.991639
4,usa,2.265625,2.187500,2.203125,2.421875,2.828125,3.031250,3.171875,3.312500,3.625000,...,24.140625,-0.687500,8.140625,1014.187500,2.734375,0.0,148.281250,2.296875,39.783730,-100.445882


In [16]:
result = instancia.unir("left", "country", "paises")

In [17]:
result.to_csv("../Ficheros/ETL2_2.csv")